In [1]:
# Import all necessary libraries
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import h5py  # Import the h5py library to read HDF5 files
from mpl_toolkits.mplot3d import Axes3D
import imageio  # Import the imageio library for creating GIFs
import os
from tempfile import TemporaryDirectory

# Define file paths
mesh_file_path = '/home/felipe/autolst/m3_L24_D12/mesh.mat'  # Path to the mesh.mat file
flow_files_path_template = '/home/felipe/autolst/m3_L24_D12/flow_{:010d}.mat'  # Template for flow_*.mat files

In [2]:
# Load the mesh.mat file
mesh_data = sio.loadmat(mesh_file_path, struct_as_record=False, squeeze_me=True)

# Extract coordinates and wall information
x = np.array(mesh_data['X'], dtype=np.float64)  # X coordinates
y = np.array(mesh_data['Y'], dtype=np.float64)  # Y coordinates
z = np.array(mesh_data['Z'], dtype=np.float64)  # Z coordinates
wall = np.array(mesh_data['wall'])  # Wall information

# Create a 3D grid using the coordinates
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

# Access nested structures in MATLAB format
flowType = mesh_data['flowType']  # Nested structure
flowParameters = mesh_data['flowParameters']  # Nested structure

# Extract x1 and x2 from flowType.cav{1, 1}.x
x1 = flowType.cav.x[0]  # Equivalent to flowType.cav{1, 1}.x(1) in MATLAB
x2 = flowType.cav.x[1]  # Equivalent to flowType.cav{1, 1}.x(2) in MATLAB
y1 = flowType.cav.y[1]  # Equivalent to flowType.cav{1, 1}.y(1) in MATLAB

# Extract Reynolds number (Re) and Mach number (Ma) from flowParameters
Re = flowParameters.Re  # Reynolds number
Ma = flowParameters.Ma  # Mach number
mu = 1 / Re  # Dynamic viscosity
nu = 1 / Re  # Kinematic viscosity

# Find the index of X and Y
indx2 = np.argmin(np.abs(x - x2))  # Index of the closest value to x2 in X
indx1 = np.argmin(np.abs(x - x1))  # Index of the closest value to x1 in X
indy1 = np.argmin(np.abs(y - y1))  # Index of the closest value to y1 in Y

# Define xend
xend = 800  # End position

# Constants
gamma = 1.4  # Specific heat ratio
T_inf = 300  # Free-stream temperature
rho_inf = 1  # Free-stream density
U_inf = 1    # Free-stream velocity

In [3]:
# Function to calculate dilatation (divergence of the velocity field)
def calculate_dilatation(U, V, W, x, y, z):
    # Calculate gradients using explicit coordinates
    dU_dx = np.gradient(U, x, axis=0)  # Gradient along X
    dV_dy = np.gradient(V, y, axis=1)  # Gradient along Y
    dW_dz = np.gradient(W, z, axis=2)  # Gradient along Z
    return dU_dx + dV_dy + dW_dz

# Create a temporary directory to store GIF frames
with TemporaryDirectory() as temp_dir:
    frame_files = []  # List to store paths of temporary frame files

    # Loop to load flow_*.mat files and process them
    for i in range(3):  # Example with 3 files (adjust as needed)
        flow_file_path = flow_files_path_template.format(i)  # Generate file name
        
        # Load the HDF5 file using h5py
        with h5py.File(flow_file_path, 'r') as flow_data:
            # Extract variables U, V, W
            U = np.array(flow_data['U']).T  # Velocity in X
            V = np.array(flow_data['V']).T  # Velocity in Y
            W = np.array(flow_data['W']).T  # Velocity in Z
            
            # Calculate dilatation
            dilatation = calculate_dilatation(U, V, W, x, y, z)
            
            # Define slice indices along the Z-axis
            z_indices = [len(z)//2]  # Slice indices [len(z)//2]
            
            # Desired limits for the X and Y axes
            x_min, x_max = -10, 800
            y_min, y_max = -12, 100
            
            # Plot colorplots for each slice and save as GIF frames
            for idx in z_indices:
                # Extract the slice at plane Z[idx]
                dilatation_slice = dilatation[:, :, idx]  # Dilatation values at plane Z[idx]
                
                # Create a mask to filter data within the X and Y limits
                x_mask = (x >= x_min) & (x <= x_max)
                y_mask = (y >= y_min) & (y <= y_max)
                
                # Apply the mask to the data
                x_filtered = x[x_mask]
                y_filtered = y[y_mask]
                dilatation_filtered = dilatation_slice[np.ix_(x_mask, y_mask)]
                
                # Create the 2D plot
                plt.figure(figsize=(8, 6))  # Adjust figure size here
                plt.imshow(
                    dilatation_filtered.T,  # Transpose to align with X and Y
                    extent=[x_filtered.min(), x_filtered.max(), y_filtered.min(), y_filtered.max()],  # Axis limits
                    origin='lower',  # Origin at the bottom-left corner
                    cmap='coolwarm',  # Divergent colormap
                    aspect=4,  # Adjust aspect ratio
                    vmin=-.2, vmax=.2  # Set fixed value limits
                )
                plt.colorbar(label='Dilatation')  # Colorbar
                plt.title(f'Slice at Z = {z[idx]:.2f} - Flow {i}')
                plt.xlabel('X')
                plt.ylabel('Y')
                
                # Save the frame as a temporary file
                frame_path = os.path.join(temp_dir, f'frame_{i}_{idx}.png')
                plt.savefig(frame_path)
                plt.close()
                frame_files.append(frame_path)

    # Create the GIF from the saved frames
    gif_path = '/home/felipe/autolst/m3_L24_D12/gifs/dilatation_animation.gif'

    # Delete the previous GIF if it exists
    if os.path.exists(gif_path):
        os.remove(gif_path)

    # Create the GIF with each frame lasting 1 second and set it to loop infinitely
    with imageio.get_writer(gif_path, mode='I', duration=2.0, loop=0) as writer:
        for frame_file in frame_files:
            image = imageio.imread(frame_file)
            writer.append_data(image)

    print(f"GIF saved successfully at: {os.path.abspath(gif_path)}")

GIF saved successfully at: /home/felipe/autolst/m3_L24_D12/gifs/dilatation_animation.gif


/tmp/ipykernel_3113568/365861238.py:79: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(frame_file)
